In [23]:
from cachetools import TTLCache

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

In [5]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
OpenAI_key = os.environ.get("OPENAI_API_KEY")

In [13]:
cache = TTLCache(maxsize=100, ttl=3600)
embeddings = OpenAIEmbeddings()


def load_faiss_index(file_path):
    global cache
    cached_index = cache.get("faiss_index")
    if cached_index:
        return cached_index
    vectorstore = FAISS.load_local(file_path, embeddings,allow_dangerous_deserialization=True)
    cache["faiss_index"] = vectorstore
    return vectorstore

In [14]:
vectorstore = load_faiss_index('state_of_the_union_faiss_index.faiss')
retriever = vectorstore.as_retriever(k=4)
temperature = 0.2  # Fine-tune for more creative or conservative responses
model_name = 'gpt-3.5-turbo-1106'

In [18]:


chat = ChatOpenAI(model=model_name, temperature=temperature)

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [15]:


query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)


In [16]:
query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
       
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [24]:
conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

demo_ephemeral_chat_history = ChatMessageHistory()

In [28]:

user_input = 'What was  Vladimir Putins  action?'
demo_ephemeral_chat_history.add_user_message(user_input)

response = conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages},
)

demo_ephemeral_chat_history.add_ai_message(response["answer"])

response

{'messages': [HumanMessage(content='What was  Vladimir Putins  action?'),
  HumanMessage(content='What was  Vladimir Putins  action?'),
  AIMessage(content="Vladimir Putin's action was a premeditated and unprovoked attack on Ukraine. He rejected diplomatic efforts and thought the West and NATO wouldn't respond. He believed he could divide the world and act with impunity. However, the free world, led by the United States and its allies, has responded with a unified and powerful effort to hold Putin accountable and support Ukraine.")],
 'context': [Document(page_content='Putin’s latest attack on Ukraine was premeditated and unprovoked. \n\nHe rejected repeated efforts at diplomacy. \n\nHe thought the West and NATO wouldn’t respond. And he thought he could divide us at home. Putin was wrong. We were ready.  Here is what we did.   \n\nWe prepared extensively and carefully. \n\nWe spent months building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Afri

In [29]:
user_input = 'Tell me about Putin’s latest attack on Ukraine?'

demo_ephemeral_chat_history.add_user_message(user_input)

conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

{'messages': [HumanMessage(content='What was  Vladimir Putins  action?'),
  HumanMessage(content='What was  Vladimir Putins  action?'),
  AIMessage(content="Vladimir Putin's action was a premeditated and unprovoked attack on Ukraine. He rejected diplomatic efforts and thought the West and NATO wouldn't respond. He believed he could divide the world and act with impunity. However, the free world, led by the United States and its allies, has responded with a unified and powerful effort to hold Putin accountable and support Ukraine."),
  HumanMessage(content='Tell me about Putin’s latest attack on Ukraine?')],
 'context': [Document(page_content='Putin’s latest attack on Ukraine was premeditated and unprovoked. \n\nHe rejected repeated efforts at diplomacy. \n\nHe thought the West and NATO wouldn’t respond. And he thought he could divide us at home. Putin was wrong. We were ready.  Here is what we did.   \n\nWe prepared extensively and carefully. \n\nWe spent months building a coalition of

In [1]:
#add_ai_message(response["answer"])
# demo_ephemeral_chat_history.add_ai_message(response["answer"])    